In [42]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv

load_dotenv()

class DebateAgent:
    def __init__(self, system_message: str):
        # Initialize the model and tools
        self.model = ChatOpenAI(model="gpt-3.5-turbo")
        self.memory = MemorySaver()
        self.search = TavilySearchResults(max_results=2)
        self.tools = [self.search]
        self.agent_executor = create_react_agent(self.model, self.tools, checkpointer=self.memory)
        
        # Initialize chat history with system message
        self.chat_history = [SystemMessage(content=system_message)]
    
    def add_human_message(self, content: str):
        # Add a human message to the chat history
        self.chat_history.append(HumanMessage(content=content))
    
    def get_ai_response(self):
        # Get the AI response based on the current chat history
        config = {"configurable": {"thread_id": "abc123"}}
        final_message = ""
        
        for chunk in self.agent_executor.stream(
            {"messages": self.chat_history}, config
        ):
            final_message = chunk
        
        response = final_message['agent']['messages'][0].content
        self.chat_history.append(AIMessage(content=response))
        return response

    def get_chat_history(self):
        return self.chat_history

# Example usage
if __name__ == "__main__":
    system_message = "You are a professional debator named Sam. You are arguing against the motion that 'The internet is a force for good'. Refer to Bob's points when arguing. The messages provided to you will be Bob's arguments."
    debate_agent = DebateAgent(system_message)

    # Add a human message
    debate_agent.add_human_message("Bob: The internet is a force of good because it enables global communication, provides access to information, fosters innovation, and connects people, facilitating collaboration and knowledge sharing across diverse communities.")

    # Get the AI response
    ai_response = debate_agent.get_ai_response()
    print("AI Response:", ai_response)
    
    # Print chat history
    for message in debate_agent.get_chat_history():
        print(message)

AI Response: As a professional debater, I disagree with the notion that the internet is solely a force for good. While it is true that the internet has the potential to bring about positive outcomes such as global communication and access to information, it also has significant drawbacks and negative consequences that cannot be overlooked.

1. **Global Communication:** While the internet does enable global communication, it also serves as a platform for spreading misinformation, hate speech, and propaganda. The ease of communication can be exploited by malicious actors to manipulate public opinion and incite violence.

2. **Access to Information:** The vast amount of information available on the internet is a double-edged sword. While it provides access to knowledge and resources, it also makes it difficult to discern credible sources from misinformation. This can lead to the spread of fake news and conspiracy theories, undermining the trust in reliable information sources.

3. **Foste